In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
import matplotlib.pyplot as plt
import sys; sys.path.insert(0, '../')
import geoplot.crs as ccrs
import geoplot as gplt
%matplotlib inline

In [3]:
# Boroughs
boroughs = gpd.read_file("../data/nyc_boroughs/boroughs.geojson", driver='GeoJSON')
boroughs['Population'] = [378, 1951, 2300, 1487, 1203]
boroughs['Population Density'] = boroughs['Population'] / boroughs['Shape_Area']
boroughs = boroughs.set_index('BoroName')
boroughs.index = [b.upper() for b in boroughs.index]

# Collisions
cdata = pd.read_csv("../data/nyc_collisions/NYPD_Motor_Vehicle_Collisions.csv", index_col=0)
def pointify(srs):
    lat, long = srs['LATITUDE'], srs['LONGITUDE']
    if pd.isnull(lat) or pd.isnull(long):
        return Point(0, 0)
    else:
        return Point(long, lat)
collisions = gpd.GeoDataFrame(cdata.head(10000), geometry=cdata.head(10000).apply(pointify, axis='columns'))
collisions = collisions[collisions.geometry.map(lambda srs: not (srs.x == 0))]
del cdata

# Citibike (Network)
citibike = pd.read_csv("https://raw.githubusercontent.com/ResidentMario/citibike-sankey/master/data/linesegments.csv", index_col=0)
def geometries(srs):
    start = Point(srs['Start Y'], srs['Start X'])
    end = Point(srs['End Y'], srs['End X'])
    return pd.Series({'From': start, 'To': end})
to_and_from = citibike.sample(1000).apply(geometries, axis='columns')

# Cities
cities = gpd.read_file("../data/cities/citiesx010g.shp")
cities = cities[cities['STATE'].map(lambda s: s not in ['PR', 'AK', 'HI', 'VI'])]

# Census Tracts
census_tracts = gpd.read_file("../data/nyc_census_tracts/census_tracts_2010.geojson", driver='GeoJSON')
from random import randint
census_tracts['mock_data'] = [randint(10, 1000) for _ in range(len(census_tracts))]

In [ ]:
gplt.pointplot(gpd.GeoDataFrame(geometry=to_and_from['To']))